In [10]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
#ChatPromptTemplate = template를 message로 부터 만든다.
#PromptTemplate = 단순 string을 이용해 template를 만든다 

#OutPut parser가 필요한 이유는, LLM 응답(response)를 변형해야 할 때가 있기 때문이다. (ex. 응답은 항상 text로 오기에 응답을 list로 변형)

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}. Also, What is your name?")

prompt = template.format(country_a="Mexico", country_b="Thailand")
#format의 return값은 string이다. 
chat.predict(prompt)




'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles).\n\nMy name is Assistant.'

In [14]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_b} and {country_b}. Also, What is your name?")
])
#tuple로 변화주기 

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ της Ταϊλάνδης και της Ταϊλάνδης είναι μηδέν, αφού πρόκειται για την ίδια χώρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [16]:
#OutPut parser가 필요한 이유는, LLM 응답(response)를 변형해야 할 때가 있기 때문이다. (ex. 응답은 항상 text로 오기에 응답을 list로 변형)
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
    #output parser는 parse라는 메서드를 꼭 구현해야 한다.
        items = text.strip().split(",")
        #strip은 text의 앞뒤 공백을 잘라준다 // 입력받은 string이 쉼표를 포함하고 있다면 쉼표를 기준으로 string을 잘라 array로 변환시켜준다.
        return list(map(str.strip,items))


In [21]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']